In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

pd.set_option("display.max_columns", None)

In [6]:
year = 2022
election_round = "first_round"

In [14]:
# Read full data
df = pd.read_csv(f"../data/Brazil/{year}_{election_round}.csv.zip")
df["polling_id"] = df["sg_uf"].astype(str) + "-" + df["nm_municipio"].astype(str) \
    + "-" + df["nr_zona"].astype(str)

df = df[df["ds_cargo"] == "Presidente"]
df.head()

,ds_cargo,cd_municipio,nm_municipio,nm_candidato,nr_candidato,sg_partido,nm_regiao,ds_sit_totalizacao,nr_turno,sg_uf,nr_zona,qt_votos_nominais,qt_votos_validos,dt_carga,polling_id
9241784,Presidente,19,GUAJARÁ-MIRIM,CIRO FERREIRA GOMES,12,PDT,NORTE,Não Eleito,1,RO,1,423,20555,2023-01-10 18:46:27,RO-GUAJARÁ-MIRIM-1
9241785,Presidente,19,GUAJARÁ-MIRIM,JAIR MESSIAS BOLSONARO,22,PL,NORTE,Segundo turno,1,RO,1,12157,20555,2023-01-10 18:46:27,RO-GUAJARÁ-MIRIM-1
9241786,Presidente,19,GUAJARÁ-MIRIM,JOSE MARIA EYMAEL,27,DC,NORTE,Não Eleito,1,RO,1,2,20555,2023-01-10 18:46:27,RO-GUAJARÁ-MIRIM-1
9241787,Presidente,19,GUAJARÁ-MIRIM,KELMON LUIS DA SILVA SOUZA,14,PTB,NORTE,Não Eleito,1,RO,1,11,20555,2023-01-10 18:46:27,RO-GUAJARÁ-MIRIM-1
9241788,Presidente,19,GUAJARÁ-MIRIM,LEONARDO PÉRICLES VIEIRA ROQUE,80,UP,NORTE,Não Eleito,1,RO,1,7,20555,2023-01-10 18:46:27,RO-GUAJARÁ-MIRIM-1


In [15]:
# df.ds_cargo.unique()

In [16]:
tt = df.groupby(["polling_id", "nm_candidato"]).agg({"qt_votos_nominais": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "nm_candidato", "qt_votos_nominais", "rate"]]

tt["rank"] = tt.groupby(["polling_id"])["qt_votos_nominais"].rank("min", ascending=False).astype(int)

In [17]:
df_filtered = tt.copy()
df_filtered = df_filtered.rename(columns={"qt_votos_nominais": "value", "nm_candidato": "candidate"})

df_filtered.head()

,polling_id,candidate,value,rate,rank
0,AC-ACRELÂNDIA-8,CIRO FERREIRA GOMES,117,0.016437,4
1,AC-ACRELÂNDIA-8,JAIR MESSIAS BOLSONARO,5238,0.735881,1
2,AC-ACRELÂNDIA-8,JOSE MARIA EYMAEL,0,0.000000,9
3,AC-ACRELÂNDIA-8,KELMON LUIS DA SILVA SOUZA,6,0.000843,7
4,AC-ACRELÂNDIA-8,LEONARDO PÉRICLES VIEIRA ROQUE,0,0.000000,9


In [18]:
df_filtered.to_csv(f"../data_output/Brazil/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [19]:
df_location = df[["sg_uf", "nr_zona", "nm_municipio", "qt_votos_validos", "polling_id"]].drop_duplicates()

df_location = df_location.rename(columns={
    "sg_uf": "region_id",
    "nm_municipio": "commune",
    "nr_zona": "polling_place",
    "qt_votos_validos": "electors"
})

df_location.to_csv(f"../data_output/Brazil/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)